# _Alchemify_ a repartitioned hybrid system

In [2]:
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys

generate a capped alanine and a system generator

In [3]:
atp, sys_gen = generate_atp()

DEBUG:perses.forcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


generate a repartitioned htf at the lambda 0 endstate

In [6]:
vac_rep_htf = generate_dipeptide_top_pos_sys(atp.topology, 
                                         new_res = 'THR', 
                                         system = atp.system, 
                                         positions = atp.positions,
                                         system_generator = sys_gen, 
                                         conduct_htf_prop=True,
                                         repartitioned=True, endstate=0, validate_endstate_energy=False)

INFO:proposal_generator:	Conducting polymer point mutation proposal...
INFO:proposal_generator:Using matching_criterion to chose best atom map
INFO:proposal_generator:Scaffold has symmetry of 0
INFO:proposal_generator:len [{4: 4, 9: 9}]
INFO:proposal_generator:{4: 4, 9: 9}
INFO:proposal_generator:Only one map so returning that one
INFO:proposal_generator:{4: 4, 9: 9}
INFO:geometry:propose: performing forward proposal
INFO:geometry:propose: unique new atoms detected; proceeding to _logp_propose...
INFO:geometry:Conducting forward proposal...
INFO:geometry:Computing proposal order with NetworkX...
INFO:geometry:number of atoms to be placed: 7
INFO:geometry:Atom index proposal order is [18, 14, 13, 17, 15, 16, 19]
INFO:geometry:omitted_bonds: []
INFO:geometry:direction of proposal is forward; creating atoms_with_positions and new positions from old system/topology...
INFO:geometry:creating growth system...
INFO:geometry:	creating bond force...
INFO:geometry:	there are 11 bonds in referenc

making topology proposal
generating geometry engine
making geometry proposal from ALA to THR
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU


INFO:geometry:	beginning construction of no_nonbonded final system...
INFO:geometry:	initial no-nonbonded final system forces ['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce']
INFO:geometry:	final no-nonbonded final system forces dict_keys(['HarmonicBondForce', 'HarmonicAngleForce', 'PeriodicTorsionForce', 'NonbondedForce'])
INFO:geometry:	there are 11 bond forces in the no-nonbonded final system
INFO:geometry:	there are 43 angle forces in the no-nonbonded final system
INFO:geometry:	there are 72 torsion forces in the no-nonbonded final system
INFO:geometry:forward final system defined with 0 neglected angles.
INFO:geometry:total reduced potential before atom placement: 16.81404844848707
INFO:geometry:total reduced energy added from growth system: -68.9733407661905
INFO:geometry:final reduced energy -52.15929231770344
INFO:geometry:sum of energies: -52.15929231770343
INFO:geometry:magnitude of difference in the energies: 1.4210854715202004e-14
INFO:g

conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.17505500214543054), ('CustomAngleForce', 11.458612091922479), ('CustomTorsionForce', 11.835112642027186), ('CustomBondForce', -92.4421205022856)]
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: CPU
conducting subsequent work with the following platform: Reference
conducting subsequent work with the following platform: CPU
added energy components: [('CustomBondForce', 0.0), ('CustomAngleFor

extract the hybrid system (this is identical to the `HTF._hybrid_system` at the lambda 0 endstate)

In [8]:
hybrid_sys = vac_rep_htf._hybrid_system

Now, we can attempt to _alchemify_ a region of interest. for the sake of this, I will just alchemify the `unique_new_atoms`. Generally, though, it might be advisable to _alchemify_ all of the atoms within a certain radius of the mutated residue

In [14]:
atoms_to_alchemify = vac_rep_htf._atom_classes['unique_new_atoms']

In [23]:
from openmmtools.alchemy import AbsoluteAlchemicalFactory, AlchemicalRegion, AlchemicalState

In [21]:
alch_factory = AbsoluteAlchemicalFactory(consistent_exceptions=False)
reference_system = hybrid_sys
alchemical_region = AlchemicalRegion(alchemical_atoms=list(atoms_to_alchemify), annihilate_electrostatics=True, annihilate_sterics=True, alchemical_torsions=True)
alchemical_system = alch_factory.create_alchemical_system(reference_system, alchemical_region)


DEBUG:openmmtools.alchemy:Dictionary of interacting alchemical regions: frozenset()
DEBUG:openmmtools.alchemy:Using 1 alchemical regions
DEBUG:openmmtools.alchemy:Adding steric interaction groups between  and the environment.
DEBUG:openmmtools.alchemy:Adding a steric interaction group between group  and .
DEBUG:openmmtools.alchemy:Adding electrostatic interaction groups between  and the environment.
DEBUG:openmmtools.alchemy:Adding a electrostatic interaction group between group  and .
DEBUG:openmmtools.utils:Create alchemically modified system took    0.021s


note that above, I also specified an annihilation of the sterics, electrostatics, and alchemical torsions of the alchemical region.

let's make an alchemical state. 

In [24]:
alchemical_state = AlchemicalState.from_system(alchemical_system)

and you'll notice that there are now `lambda` parameters for torsions, sterics, and electrostatics!

In [27]:
vars(alchemical_state)

{'_function_variables': {},
 '_parameters': {'lambda_bonds': None,
  'lambda_angles': None,
  'lambda_torsions': 1.0,
  'lambda_sterics': 1.0,
  'lambda_electrostatics': 1.0},
 '_parameters_name_suffix': None}

I'll now demonstrate how to actually create a context and run MD with the alchemical system...

In [47]:
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from simtk import openmm, unit
from openmmtools.integrators import LangevinIntegrator

T = 300 * unit.kelvin

In [32]:
thermostate = ThermodynamicState(system = alchemical_system, temperature = T)
comp_thermostate = CompoundThermodynamicState(thermodynamic_state=thermostate, composable_states=[alchemical_state])


we can query the lambdas...

In [33]:
comp_thermostate.lambda_sterics

1.0

let's say we want to start the system by cutting the nonbonded terms in half


In [34]:
comp_thermostate.lambda_sterics = 0.5
comp_thermostate.lambda_electrostatics = 0.5

In [40]:
integrator = LangevinIntegrator(temperature = T, timestep = 2.0 * unit.femtoseconds)

In [41]:
context = comp_thermostate.create_context(integrator) #the compound thermostate and the integrator need the same Temp

now you can directly query the global parameters in the context...

In [45]:
swig_parameters = context.getParameters()
context_parameters = {q: swig_parameters[q] for q in swig_parameters}
print(context_parameters)

{'lambda_electrostatics': 0.5, 'lambda_sterics': 0.5, 'lambda_torsions': 1.0, 'softcore_a': 1.0, 'softcore_alpha': 0.5, 'softcore_b': 1.0, 'softcore_beta': 0.0, 'softcore_c': 6.0, 'softcore_d': 1.0, 'softcore_e': 1.0, 'softcore_f': 2.0}


neat-o <br>
let's run some MD

In [50]:
sampler_state = SamplerState(positions = vac_rep_htf._hybrid_positions)
sampler_state.apply_to_context(context)
context.setVelocitiesToTemperature(T)

In [51]:
integrator.step(10)

we just ran 10 md steps with softened sterics and electrostatics...let's go back to the original system by resetting the lambda-nonbondeds and apply the compound thermostate to the context...

In [52]:
comp_thermostate.lambda_sterics = 1.
comp_thermostate.lambda_electrostatics = 1.

comp_thermostate.apply_to_context(context)

let's run 10 more steps of MD...

In [53]:
integrator.step(10)

let's extract the positions to the sampler state again...

In [54]:
sampler_state.update_from_context(context)

## Next steps:
notice now that there are several neat tricks we can try...
First, we can perform BLUES, which works like this: 
1. alternate MD steps as you decrement the values of any of the lambdas from 1.0 to your choosing (collecting work along the way)...
2. once the target lambda values have been achieved, you have the choice of performing a rigid body rotation (or any proposal kernel s.t. the forward probability kernel can be computed), at which time, the reverse annealing protocol (back to lambda=1) can be performed.
3. the whole move is accepted or rejected 

we can also do things like repex where we compose several alchemical windows, run replica exchange, and extract the snapshots at the first window (lambda=1) which samples the Boltzmann at out alchemical endstate of interest.

The _real_ next step is to do a little playing around with the alchemical system nonbonded force so that we can perform REST. I will pursue this soon.